<a href="https://colab.research.google.com/github/nhgowtham/hackathon_gg/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset download

In [ ]:
# https://drive.google.com/drive/folders/1u0z4pkZJN8Q00zt_QyEWHpckuFJ_z3iZ?usp=drive_link

#  Graphere dataset
!gdown --folder https://drive.google.com/drive/folders/1u0z4pkZJN8Q00zt_QyEWHpckuFJ_z3iZ -O data/


# https://drive.google.com/drive/folders/1DTMrT_Ihpuopwny63f4OwjwPTPtXmiy2?usp=drive_link
# CoAg
!gdown --folder https://drive.google.com/drive/folders/1DTMrT_Ihpuopwny63f4OwjwPTPtXmiy2 -O data/


# https://drive.google.com/drive/folders/18BkhR-fbuuavqN7LAtrvWeJmxP-p2WXS?usp=drive_link
#MoS2_Nanowire
!gdown --folder https://drive.google.com/drive/folders/18BkhR-fbuuavqN7LAtrvWeJmxP-p2WXS -O data/

# https://drive.google.com/drive/folders/1mkY9KlfnZXsvFfHLFujNmMPx4YTE0yxD?usp=drive_link
# MoS2 monolayer
!gdown --folder https://drive.google.com/drive/folders/1mkY9KlfnZXsvFfHLFujNmMPx4YTE0yxD -O data/

# https://drive.google.com/drive/folders/1qUwUopeyzAXqVQ3ROs3XEBmW9yY9Yrn6?usp=drive_link
# EELS
!gdown --folder https://drive.google.com/drive/folders/1qUwUopeyzAXqVQ3ROs3XEBmW9yY9Yrn6 -O data/


# https://drive.google.com/drive/folders/1lwFqZnVGk0qjoRYeKAJoNuI90vm7Nnos?usp=drive_link
# SnSe dataset
!gdown --folder https://drive.google.com/drive/folders/1lwFqZnVGk0qjoRYeKAJoNuI90vm7Nnos -O data/

In [ ]:

!pip install -q pyro5
!pip install -q scifireaders
!pip install -q sidpy
!pip install -q pynsid
!pip install -q git+https://github.com/pycroscopy/DTMicroscope.git
!pip install pytemlib
!pip install -q git+https://github.com/pycroscopy/DTMicroscope.git@utk
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install torch torchvision
!pip install matplotlib



In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

# Path to the combined dataset file
combined_h5_path = "combined_dataset.h5"

def list_all_datasets(h5file):
    paths = []
    def visitor(name, obj):
        if isinstance(obj, h5py.Dataset):
            paths.append(name)
    h5file.visititems(visitor)
    return paths

def extract_preview(dataset):
    arr = dataset[()]
    while arr.ndim > 2:
        arr = arr[arr.shape[0] // 2]
    return np.squeeze(arr)

def show_image(image, title):
    plt.figure(figsize=(4, 4))
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Open and visualize
with h5py.File(combined_h5_path, "r") as f:
    all_datasets = list_all_datasets(f)

    for path in all_datasets:
        try:
            dset = f[path]
            shape = dset.shape
            if dset.ndim >= 2:
                preview = extract_preview(dset)
                print(f"[OK] {path} → shape = {shape}")
                show_image(preview, f"{path}\nshape={shape}")
        except Exception as e:
            print(f"[ERROR] {path}: {e}")


### Data preprocessing

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms
from dinov2.data.transforms import make_classification_eval_transform
import os

# Constants
DINO_MODEL = "dinov2_vitb14"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PATCH_SIZE = 224
STRIDE = 112
H5_PATH = "combined_dataset.h5"
OUTPUT_H5 = "dinov2_embeddings.h5"

# Load DINOv2 model and transform
model = torch.hub.load("facebookresearch/dinov2", DINO_MODEL).to(DEVICE).eval()
transform = make_classification_eval_transform()

def normalize_percentile(img, pmin=2, pmax=98):
    lo, hi = np.percentile(img, (pmin, pmax))
    return np.clip((img - lo) / (hi - lo), 0, 1)

def tile_image(image, patch_size=PATCH_SIZE, stride=STRIDE):
    h, w = image.shape
    patches, positions = [], []
    for i in range(0, h - patch_size + 1, stride):
        for j in range(0, w - patch_size + 1, stride):
            patch = image[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
            positions.append((i, j))
    return patches, positions

def extract_embeddings(patches: list[np.ndarray], model, transform):
    if len(patches) == 0:
        return []
    tensor_batch = torch.stack([
        transform(Image.fromarray((p * 255).astype(np.uint8)).convert("RGB")) for p in patches
    ])
    with torch.no_grad():
        feats = model.forward_features(tensor_batch.to(DEVICE))["x_norm_patchtokens"]
        embeddings = feats.mean(dim=1).cpu().numpy()
    return embeddings

def process_h5_image(image_array):
    img = normalize_percentile(image_array)
    patches, positions = tile_image(img)
    if len(patches) == 0:
        raise ValueError("No patches extracted from image")
    embeddings = extract_embeddings(patches, model, transform)
    return embeddings, positions, img

def extract_preview(dataset):
    try:
        arr = np.array(dataset)
        while arr.ndim > 2:
            arr = arr[arr.shape[0] // 2]
        return np.squeeze(arr).astype(np.float32)
    except Exception as e:
        print(f"[extract_preview ERROR] {dataset.name}: {e}")
        raise

def list_all_valid_datasets(h5file):
    paths = []
    def visitor(name, obj):
        if isinstance(obj, h5py.Dataset) and obj.ndim == 2 and min(obj.shape) >= PATCH_SIZE:
            paths.append(name)
    h5file.visititems(visitor)
    return paths

# Save embeddings and positions
def save_to_hdf5(h5_file, group_name, embeddings, positions, image):
    grp = h5_file.require_group(group_name)
    grp.create_dataset("embeddings", data=embeddings, compression="gzip")
    grp.create_dataset("positions", data=np.array(positions), compression="gzip")
    grp.create_dataset("image", data=image.astype(np.float32), compression="gzip")  # <- Add image save


# Main loop
with h5py.File(H5_PATH, "r") as f_in, h5py.File(OUTPUT_H5, "a") as f_out:
    valid_paths = list_all_valid_datasets(f_in)

    for path in valid_paths:
        try:
            dset = f_in[path]
            img = extract_preview(dset)

            print(f"[PROCESSING] {path} with shape {img.shape}")
            emb, pos, img_norm = process_h5_image(img)
            save_to_hdf5(f_out, path.replace("/", "_"), emb, pos, img_norm)
            print(f"Saved {len(emb)} embeddings for {path}")
        except Exception as e:
            print(f"[ERROR] {path}: {e}")


In [ ]:
import h5py
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

EMB_H5 = "dinov2_embeddings.h5"
KEYWORD = "Graphene"  # <-- Change to "MoS2" or "Graphene" etc.

def load_embeddings_for_subset(h5_path, keyword):
    embeddings = []
    labels = []
    with h5py.File(h5_path, "r") as f:
        for key in f:
            if keyword.lower() in key.lower():
                data = f[key]["embeddings"][()]
                embeddings.append(data)
                labels.extend([key] * len(data))
    if embeddings:
        all_embeddings = np.concatenate(embeddings, axis=0)
        return all_embeddings, labels
    else:
        raise ValueError(f"No entries found for keyword: {keyword}")

# Load + PCA
X, labels = load_embeddings_for_subset(EMB_H5, KEYWORD)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Plot
plt.figure(figsize=(6, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], s=5, alpha=0.7)
plt.title(f"PCA of DINOv2 Embeddings - {KEYWORD}")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()


In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import umap
import os


# === CONFIGURATION ===
H5_PATH = "dinov2_embeddings.h5"  # Embeddings saved here by earlier step
OUTPUT_DIR = "viz_outputs"
TARGET_MATERIAL = "SnSe"  # or 'MoS2', 'Graphene', etc.
N_COMPONENTS = 2
N_CLUSTERS = 4
os.makedirs(OUTPUT_DIR, exist_ok=True)


def load_embeddings_for_material(h5_path, material_key):
    all_embeds = []
    all_positions = []
    with h5py.File(h5_path, 'r') as f:
        for name in f.keys():
            if material_key.lower() in name.lower():
                emb = f[name]["embeddings"][:]
                pos = f[name]["positions"][:]
                all_embeds.append(emb)
                all_positions.append(pos)
    return np.vstack(all_embeds), np.vstack(all_positions)


def run_dimensionality_reduction(embeddings, method='pca'):
    if method == 'pca':
        reducer = PCA(n_components=N_COMPONENTS)
    elif method == 'tsne':
        reducer = TSNE(n_components=N_COMPONENTS, perplexity=30, n_iter=500)
    elif method == 'umap':
        reducer = umap.UMAP(n_components=N_COMPONENTS)
    else:
        raise ValueError("Unknown reduction method")
    return reducer.fit_transform(embeddings)


def run_clustering(embeddings, n_clusters=N_CLUSTERS):
    model = KMeans(n_clusters=n_clusters, random_state=42)
    return model.fit_predict(embeddings)


def plot_reduced(reduced, labels, title, filename):
    plt.figure(figsize=(6, 6))
    scatter = plt.scatter(reduced[:, 0], reduced[:, 1], c=labels, cmap='tab10', s=5, alpha=0.7)
    plt.title(title)
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.colorbar(scatter)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, filename))
    plt.show()


def main():
    print(f"Loading embeddings for material: {TARGET_MATERIAL}")
    emb, pos = load_embeddings_for_material(H5_PATH, TARGET_MATERIAL)

    print("Reducing dimensions with PCA and UMAP...")
    reduced_pca = run_dimensionality_reduction(emb, method='pca')
    reduced_umap = run_dimensionality_reduction(emb, method='umap')

    print("Running K-means clustering...")
    cluster_labels = run_clustering(emb)

    print("Plotting...")
    plot_reduced(reduced_pca, cluster_labels,
                 title=f"PCA Clusters - {TARGET_MATERIAL}",
                 filename=f"{TARGET_MATERIAL}_pca_clusters.png")

    plot_reduced(reduced_umap, cluster_labels,
                 title=f"UMAP Clusters - {TARGET_MATERIAL}",
                 filename=f"{TARGET_MATERIAL}_umap_clusters.png")

    print("Done.")


if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import h5py
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import umap
import os

# --- Load Embeddings ---
EMBED_H5_PATH = "dinov2_embeddings.h5"
SUBSTR_FILTER = "SnSe"  # <-- Change this to "Graphene" or "MoS2" etc.

with h5py.File(EMBED_H5_PATH, 'r') as f:
    all_keys = [k for k in f.keys() if SUBSTR_FILTER in k]
    all_embeds = np.concatenate([f[k]['embeddings'][()] for k in all_keys])
    all_positions = np.concatenate([f[k]['positions'][()] for k in all_keys])
    img_shape = f[all_keys[0]]['image'].shape
    img = f[all_keys[0]]['image'][()]

# --- Dimensionality Reduction + Clustering ---
reducer = umap.UMAP(n_components=2, random_state=42)
umap_proj = reducer.fit_transform(all_embeds)

kmeans = KMeans(n_clusters=4, random_state=0).fit(umap_proj)
labels = kmeans.labels_

# --- Plot ROIs on Original Image ---
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(img, cmap='gray')
for (y, x), label in zip(all_positions, labels):
    rect = patches.Rectangle((x, y), PATCH_SIZE, PATCH_SIZE,
                             linewidth=0.5, edgecolor=plt.cm.tab10(label), facecolor='none')
    ax.add_patch(rect)
ax.set_title(f"Clusters mapped on image - {SUBSTR_FILTER}")
plt.axis('off')
plt.tight_layout()

plt.show()


In [ ]:
from PIL import Image

output_dir = f"patch_clusters_{SUBSTR_FILTER}"
os.makedirs(output_dir, exist_ok=True)

for (y, x), label in zip(all_positions, labels):
    patch = img[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
    fname = f"{output_dir}/cluster{label}_{y}_{x}.png"
    Image.fromarray((patch * 255).astype(np.uint8)).save(fname)


In [ ]:
import os
import h5py
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
from sklearn.cluster import KMeans
from torchvision import transforms
from dinov2.data.transforms import make_classification_eval_transform
from skimage.feature import blob_log

# === CONFIG ===
h5_path = "/home/gowthamnh/hackathon_gg/data/MoS2_Nanowire/kmr_doped/HAADF_4.h5"  # Change this to the .h5 file
PATCH_SIZE = 64
DINO_RESIZE = 224
N_CLUSTERS = 4
BATCH_SIZE = 256
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DINO_MODEL = "dinov2_vitb14"
OUTPUT_H5 = "atom_embeddings_individual.h5"

# === Load DINOv2 ===
model = torch.hub.load("facebookresearch/dinov2", DINO_MODEL).to(DEVICE).eval()
transform = make_classification_eval_transform()

def normalize_percentile(img, pmin=2, pmax=98):
    lo, hi = np.percentile(img, (pmin, pmax))
    return np.clip((img - lo) / (hi - lo), 0, 1)

def extract_first_2d_dataset(g):
    if isinstance(g, h5py.Dataset) and g.ndim >= 2:
        arr = g[()]
        while arr.ndim > 2:
            arr = arr[arr.shape[0] // 2]
        return np.squeeze(arr).astype(np.float32)
    elif isinstance(g, h5py.Group):
        for name in g:
            try:
                return extract_first_2d_dataset(g[name])
            except Exception:
                continue
    raise ValueError("No valid 2D dataset found.")

def detect_atoms_log(img, min_sigma=1.5, max_sigma=3.5, threshold=0.02):
    blobs = blob_log(img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold)
    return [(int(y), int(x)) for y, x, _ in blobs]

def crop_patch(img, coords):
    patches, valid_coords = [], []
    h, w = img.shape
    half = PATCH_SIZE // 2
    for (y, x) in coords:
        if y - half < 0 or y + half > h or x - half < 0 or x + half > w:
            continue
        patch = img[y - half:y + half, x - half:x + half]
        patch = cv2.resize(patch, (DINO_RESIZE, DINO_RESIZE), interpolation=cv2.INTER_CUBIC)
        tensor = transform(Image.fromarray((patch * 255).astype(np.uint8)).convert("RGB"))
        patches.append(tensor)
        valid_coords.append((y, x))
    return patches, valid_coords

# === Run processing ===
fname = os.path.splitext(os.path.basename(h5_path))[0]
with h5py.File(h5_path, 'r') as f_in:
    img = extract_first_2d_dataset(f_in)
    img = normalize_percentile(img)
    print(f"[PROCESSING] {h5_path} with shape {img.shape}")

    coords = detect_atoms_log(img)
    if len(coords) == 0:
        print("[SKIP] No atoms found.")
    else:
        all_embeddings = []
        all_coords = []
        patch_batch, valid_coords = crop_patch(img, coords)

        for i in range(0, len(patch_batch), BATCH_SIZE):
            batch = torch.stack(patch_batch[i:i + BATCH_SIZE]).to(DEVICE)
            with torch.no_grad():
                feats = model.forward_features(batch)["x_norm_patchtokens"]
                emb = feats.mean(dim=1).cpu().numpy()
            all_embeddings.append(emb)
            all_coords.extend(valid_coords[i:i + BATCH_SIZE])

        all_embeddings = np.vstack(all_embeddings)

        with h5py.File(OUTPUT_H5, "a") as f_out:
            group = f_out.require_group(fname)
            group.create_dataset("embeddings", data=all_embeddings)
            group.create_dataset("coords", data=np.array(all_coords))
            group.create_dataset("image", data=img.astype(np.float32))
        print(f"[✓] Saved {len(all_embeddings)} embeddings for {fname}")

        kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42)
        labels = kmeans.fit_predict(all_embeddings)

        fig, ax = plt.subplots(figsize=(8, 8))
        ax.imshow(img, cmap="viridis")
        for (y, x), label in zip(all_coords, labels):
            circ = patches.Circle((x, y), radius=4, color=plt.cm.tab10(label), fill=True, linewidth=0)
            ax.add_patch(circ)
        ax.set_title(f"Clustered Atoms - {fname}")
        ax.axis("off")
        plt.tight_layout()
        plt.show()


### Dataset CdTe_STO

In [ ]:
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
from tifffile import imread

# === Settings ===
ROOT_DIR = "/home/gowthamnh/hackathon_gg/data/image_data"
DATASETS = {
    "cdte_dataset.h5": "DataSet_CdTe",
    "sto_dataset.h5": "DataSet_STO"
}
N_PREVIEW = 5  # For visual inspection

# === Utils ===
def normalize_percentile(img, pmin=2, pmax=98):
    lo, hi = np.percentile(img, (pmin, pmax))
    return np.clip((img - lo) / (hi - lo), 0, 1)

def extract_label(fname):
    return os.path.basename(fname).split("_")[0].lower()

def find_all_tif_files(folder):
    tif_files = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith(".tif"):
                tif_files.append(os.path.join(root, file))
    return tif_files

# === Core Processing ===
def process_dataset(out_h5, subfolder):
    input_dir = os.path.join(ROOT_DIR, subfolder)
    all_files = find_all_tif_files(input_dir)
    print(f"📁 Processing {subfolder} ({len(all_files)} images)")

    label_map = {}
    key_list = []
    previews = []

    with h5py.File(out_h5, "w") as f:
        image_group = f.create_group("images")
        label_ds = f.create_group("labels")

        for path in all_files:
            try:
                raw = imread(path)
                if raw.ndim > 2:
                    raw = raw[..., 0]
                raw = np.squeeze(raw).astype(np.float32)
                norm = normalize_percentile(raw)

                fname = os.path.basename(path)
                label = extract_label(fname)
                key = os.path.splitext(fname)[0].lower()

                # Avoid duplicates
                if key in image_group:
                    i = 1
                    while f"{key}_{i}" in image_group:
                        i += 1
                    key = f"{key}_{i}"

                image_group.create_dataset(key, data=norm, compression="gzip")
                label_ds.attrs[key] = label
                key_list.append(key)

                label_map[label] = label_map.get(label, 0) + 1

                if len(previews) < N_PREVIEW:
                    previews.append((raw, norm, label, fname))

            except Exception as e:
                print(f"[ERROR] {path}: {e}")

        f.create_dataset("image_keys", data=np.array(key_list, dtype='S'))

    print(f"✅ Saved to {out_h5}")
    print("📊 Label summary:", label_map)
    return previews

# === Plot ===
def plot_previews(previews, title):
    fig, axs = plt.subplots(len(previews), 2, figsize=(6, 2 * len(previews)))
    for i, (raw, norm, label, fname) in enumerate(previews):
        axs[i, 0].imshow(raw, cmap='gray')
        axs[i, 0].set_title(f"Original - {fname}")
        axs[i, 1].imshow(norm, cmap='gray')
        axs[i, 1].set_title(f"Normalized - label: {label}")
    fig.suptitle(title)
    plt.tight_layout()
    plt.show()

# === Execute ===
for out_file, folder in DATASETS.items():
    previews = process_dataset(out_file, folder)
    plot_previews(previews, f"Preview from {out_file}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import h5py
from skimage.feature import blob_log
from skimage.filters import gaussian
from skimage import exposure
import random

# === Parameters ===
H5_PATH = "cdte_dataset.h5"  # or "sto_dataset.h5"
N_SAMPLES = 3
PATCH_SIZE = 64

# === Processing Functions ===

def normalize_percentile(img, pmin=2, pmax=98):
    lo, hi = np.percentile(img, (pmin, pmax))
    return np.clip((img - lo) / (hi - lo), 0, 1)

def enhance_contrast(img):
    return exposure.equalize_adapthist(img, clip_limit=0.03)

def preprocess_for_atom_detection(img):
    img_norm = normalize_percentile(img)
    img_smooth = gaussian(img_norm, sigma=1)
    img_enhanced = enhance_contrast(img_smooth)
    return img_enhanced

def detect_atoms_log(img, min_sigma=1.2, max_sigma=2.5, threshold=0.02):
    blobs = blob_log(img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold)
    return [(int(y), int(x)) for y, x, _ in blobs]

def extract_patches(norm_img, coords, patch_size=64, max_patches=3):
    patches = []
    h, w = norm_img.shape
    for (y, x) in coords[:max_patches]:
        y1 = max(y - patch_size // 2, 0)
        y2 = min(y + patch_size // 2, h)
        x1 = max(x - patch_size // 2, 0)
        x2 = min(x + patch_size // 2, w)
        patch = norm_img[y1:y2, x1:x2]
        if patch.shape == (patch_size, patch_size):
            patches.append(patch)
    return patches

# === Collage Visualization ===

def show_collage(img, norm, coords, patches, label, title):
    circle_img = np.stack([norm]*3, axis=-1)
    for (y, x) in coords:
        cv2.circle(circle_img, (x, y), 4, (1.0, 0.0, 0.0), 1)

    fig, axs = plt.subplots(2, 3, figsize=(10, 6))
    axs[0, 0].imshow(img, cmap='gray'); axs[0, 0].set_title("Original")
    axs[0, 1].imshow(norm, cmap='gray'); axs[0, 1].set_title("Normalized")
    axs[0, 2].imshow(circle_img); axs[0, 2].set_title("Atoms Detected")

    for i, patch in enumerate(patches[:3]):
        axs[1, i].imshow(patch, cmap='gray')
        axs[1, i].set_title(f"Patch {i+1}")
        axs[1, i].axis('off')

    plt.suptitle(f"{title} — Label: {label}")
    plt.tight_layout()
    plt.show()

# === Run Over H5 ===

def run_visual_pipeline(h5_file, n_samples=3):
    with h5py.File(h5_file, 'r') as f:
        image_group = f["images"]
        labels = dict(f["labels"].attrs)
        all_keys = list(image_group.keys())

        # Invert label map to {label: [keys]}
        label_to_keys = {}
        for k, v in labels.items():
            label = v.decode("utf-8") if isinstance(v, bytes) else v
            label_to_keys.setdefault(label, []).append(k)

        print(f"📊 Labels found: {list(label_to_keys.keys())}")

        for label, keys in label_to_keys.items():
            sample_keys = random.sample(keys, min(len(keys), n_samples))
            for key in sample_keys:
                img = image_group[key][()]
                norm = preprocess_for_atom_detection(img)
                coords = detect_atoms_log(norm)
                patches = extract_patches(norm, coords, PATCH_SIZE)
                show_collage(img, norm, coords, patches, label, key)

# === Example Run ===
# run_visual_pipeline("cdte_dataset.h5", n_samples=3)
run_visual_pipeline("sto_dataset.h5", n_samples=2)


In [1]:
import atomai
print(atomai.__version__)
from atomai.models import *  # this should now work


0.8.1


In [3]:
!ls data

Graphene  image_data  Mos2_monolayer  MoS2_Nanowire  SnSe


In [4]:
pwd

'/home/gowthamnh/hackathon_gg'